<a href="https://colab.research.google.com/github/yohanesnuwara/python-bootcamp-for-geoengineers/blob/master/EnP_training/session3_formation_evaluation_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Formation Evaluation with Python

In [ ]:
# import numpy, matplotlib, and pandas


In [ ]:
# install and import lasio


In [ ]:
# cloning github repository of formation-evaluation
!git clone https://github.com/yohanesnuwara/formation-evaluation

In [ ]:
# import all functions from formation-evaluation


## Functions

In [ ]:
def calculate_klogh(formation_name, phif, vsh):
  # there is no available equation for Heather Fm. 
  if formation_name == 'hugin':
    return 10 ** (2 + (8 * phif) - (9 * vsh))
  if formation_name == 'sleipner':
    return 10 ** (-3 + (32 * phif) - (2 * vsh))
  if formation_name == 'skagerak':
    return 10 ** (-1.85 + (17.4 * phif) - (3 * vsh))

## Load well log data

In [ ]:
# specify file path (well 15/9-F-11A)
filepath = '/content/formation-evaluation/data/volve/15_9-F-1A.LAS'


# read with lasio




In [ ]:
# check the available logs with .keys()


In [ ]:
# look for more detail with .curves


In [ ]:
# see the data using .data


In [ ]:
# convert it to Pandas dataframe using .df() and then reset index


In [ ]:
# then show the dataframe


## Generate formation labels from formation tops file

We have formation tops file. What we are going to do now is to give formation top labels to the logs. We'll use function `label_generator`.

In [ ]:
# specify file path
filepath = '/content/formation-evaluation/data/volve/Fmtops_15_9-F-11A.csv'

# read with Pandas


In [ ]:
# see help to find what visualization inputs are required


In [ ]:
# use "label_generator" to generate formation labels
# then show the dataframe


As we see here we have NaN in the `FM. LABEL` column simply because the depths are not corresponding to any Fm. tops. So we can replace NaNs with a string named as `Unknown`

In [ ]:
# Replace NaNs in formation label column with string called "Unknown"


## Visualize

### Well-log visualization

We will use function `well_log_display`

In [ ]:
# see help to find what visualization inputs are required
# identify which input is OBLIGATORY, which input is OPTIONAL


Use default

In [ ]:
# Logs we gonna visualize are: 'NPHI', 'RHOB', 'GR', 'RT', 'PEF', 'CALI', 'DT', 'DTS' 


Make the visualization looks better (adding the optional variables)

* Transform the resistivity axis from Cartesian axis to Semilog axis
* Give limits to the x and depth axes
* Give different colors to each log

In [ ]:
# adding optional variables like "min_depth", "max_depth", etc.


### Triple combo

We are seeking hydrocarbon occurence in depth between 3,580 to 3,702 m. So, we'll visualize the crossover in a triple combo. We will use function `triple_combo`

In [ ]:
# see help to find what visualization inputs are required
# identify which input is OBLIGATORY, which input is OPTIONAL


Use default

In [ ]:
# triple combo at depth from 3,580 to 3,702 m


Make the visualization looks better (adding the optional variables)

* Give limits to the x axes (specifically, GR and resistivity)
* Give a sand GR line, e.g. 50 API
* Adjust figure size and title height (if needed)

In [ ]:
# adding optional variables like "sand_GR_line", "figsize", etc.


### Neutron Density Plot

We wish to identify the lithologies of each formation (whether sandstone, limestone, or dolomite) using ND plot. We will use function `ND_plot`.

In [ ]:
# see help to find what visualization inputs are required
# identify which input is OBLIGATORY, which input is OPTIONAL


Plot the NPHI and RHOB points differentiated by each formation names. (categorical)

In [ ]:
# use "ND_plot" to make a plot with colors based on formation names


Plot NPHI and RHOB points differentiated by depths/other continuous variables

In [ ]:
# use "ND_plot" to make a plot with colors based on continuous variables


## Petrophysical Calculation per Formation Top

You already have the data with formation labels. Next we will calculate new petrophysical variables PER FORMATION LABEL (only Heather and Hugin). 

> Sleipner will be your assignment.



First thing we need to do is to "mask the subset of dataframe in FM. LABEL column that contains each of the above names".

In [ ]:
# list all formation labels using df['x'].unique()


In [ ]:
# create masks for Heather and Hugin using string contains: str.contains.('...')



# see inside the mask


Now isolate Heather and Hugin into individual subsets of Dataframe, using the masks that we made before.

In [ ]:
# create dataframe subsets of Heather and Hugin: df[mask]



# print the Heather dataframe


We calculate new petrophysical outputs on these subsets. Based on petrophysical report, each formation has different variables like fluid density ($\rho_{fl}$), etc. The variables have been already listed below.

In [1]:
# List of individual variables of each formation (DO NOT CHANGE THESE VALUES)
#            Heath  Hugin   Sleip     
rho_ma    = [2.66,  2.65,   2.65]        
rho_fl    = [1,     0.9,    0.9]
gr_min    = [7,     7,      7]
gr_max    = [120,   150,    105]
A         = [0,     0.4,    0.4]  # regression coeff for PHID
B         = [0,     0.01,   0.01] # regression coeff for PHID

# from report, m of Hugin and Sleipner is calculated using specific formula (p. 15, 16)
# now we just assume both has the following values (taken from Asquith, p. 5)
# Hugin is "estimated" as consolidated sst, Sleipner as carbonates
m         = [2,     2,      2.14] 
a         = [1,     1,      1]
n         = [2,     2.45,   2.45]
# water resistivity, already extrapolated at depth from Rw @ 20C = 0.07 ohm-m
Rw        = [0.022, 0.022,  0.022] 

### 1. Calculate density porosity (PHID)

![image](https://user-images.githubusercontent.com/51282928/98511443-42241880-2297-11eb-94c2-c839ef7a35ba.png)

In [ ]:
# calculate PHID for Heather and Hugin


### 2. Calculate total porosity (PHIF)

In [ ]:
# calculate PHIF for Heather and Hugin
# phid + A * (NPHI - phid) + B


# display histogram with bins 10


Comment: some porosities are negative. This is normal because there are densities larger than calcite (> 2.71), while the measurement is in lst. unit check: `max(...)`

In [ ]:
# maximum value of Heather's RHOB


### 3. Calculate shale volume (VSH)

![image](https://user-images.githubusercontent.com/51282928/98511687-ae9f1780-2297-11eb-9d96-9e50631ded03.png)

In [ ]:
# calculate VSH for Heather and Hugin



# print summary stats: df.describe()


### 4. Calculate water saturation (SW)

![image](https://user-images.githubusercontent.com/51282928/98511817-ea39e180-2297-11eb-9919-50d9fb9dc4ea.png)



In [ ]:
# calculate SW for Heather and Hugin



Comment: In Heather most SW result is larger than 1! 
However this is true because they're in fact water zone.
So, we can mask all SW values larger than 1, to be changed to 1

In [ ]:
## create mask



## change anyting larger than 1 with 1 using the mask




### Calculate permeability (KLOGH)

The functions have been prepared for you. The equation used are from the report (p. 13), different for each formation.

![image](https://user-images.githubusercontent.com/51282928/98511897-089fdd00-2298-11eb-958b-3c2a05ce13e0.png)

In [ ]:
# There is no equation for Heather Fm., so Heather is not calculated


# display boxplot




Comment: there are lots of outliers of Hugin permeability

## Visualize calculations result

We have done with calculations, let's visualize them into logs. We will now practice using bare Matplotlib.

In [ ]:
# visualize calculation result as derived logs (Hugin)








## End of the training!

Now we know how to:
* Read .LAS file using `lasio` and convert to Pandas dataframe
* Generate formation top labels from formation top file .CSV 
* Visualize well-logs
* Visualize triple combo
* Visualize Neutron-Density plot (with categorical and continuous variables)
* Making subsets of dataframes based on each formation name
* Compute petrophysical variables (PHIF, SW, VSH, K)
* Visualize the results

## Copyright

`formation-evaluation` repository that stores all the functions and data, and this notebook, are copyrights of Yohanes Nuwara (2020). This notebook is contained in [this repository](https://github.com/yohanesnuwara/python-bootcamp-for-geoengineers) You may freely distribute for self-study and tutorials, but you will consider the authorship of all the codes written here. 

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.